In [1]:
from calendar import month

import gudhi
import numpy as np

from functions.data_loading import get_filtered_data_range, get_data_range, get_flight_persistence
from functions.data_filtering import complete_flight_filter

from datetime import datetime
from collections import Counter
from tqdm import tqdm
from traffic.core import Traffic, Flight

In [2]:
origin = "bergen"
destination = "oslo"

unfiltered_data, file_name = get_data_range(origin, destination, datetime(year=2023, month=1, day=1), datetime(year=2024, month=1, day=1))
filtered_flights, file_name = get_filtered_data_range(unfiltered_data, file_name, complete_flight_filter(origin, destination))
persistences, _ = get_flight_persistence(filtered_flights, file_name)

In [9]:
rows = []
for i in tqdm(filtered_flights):
    data = i.data.copy(deep=True)
    
    unwrapped = np.unwrap(np.deg2rad(data["heading"]), period=2 * np.pi, discont=np.pi)
    unwrapped_max = unwrapped.max()
    unwrapped_min = unwrapped.min()
    
    length = (data["timestamp"].max() - data["timestamp"].min()).total_seconds()
    
    max_lat = data["latitude"].max()
    min_lat = data["latitude"].min()
    max_lon = data["longitude"].max()
    min_lon = data["longitude"].min()
    
    max_velocity = data["velocity"].max()
    min_velocity = data["velocity"].min()
    
    row = [unwrapped_max, unwrapped_min, unwrapped_max-unwrapped_min, length, max_lat, min_lat, max_lon, min_lon, max_velocity, min_velocity]
    rows.append(row)

# Convert the list of rows into a NumPy array
final_array = np.array(rows)

print(final_array)

100%|██████████| 3219/3219 [00:04<00:00, 689.05it/s] 

[[3.48123394e+00 1.19276724e+00 2.28846671e+00 ... 5.21973709e+00
  1.72102087e+02 5.99233416e+01]
 [2.98050075e+00 2.68703025e-01 2.71179772e+00 ... 5.22153657e+00
  2.23253286e+02 6.93946629e+01]
 [3.42853141e+00 1.19163766e+00 2.23689375e+00 ... 5.22115435e+00
  2.20672491e+02 6.47995263e+01]
 ...
 [2.98236808e+00 2.25166196e-01 2.75720188e+00 ... 5.22172599e+00
  2.24097499e+02 5.37095304e+01]
 [3.43518164e+00 1.18954701e+00 2.24563463e+00 ... 5.21801540e+00
  2.27945719e+02 7.01702370e+01]
 [3.44987601e+00 1.20537176e+00 2.24450425e+00 ... 5.21997724e+00
  2.41190309e+02 6.47995263e+01]]


In [15]:
final_array[:, 2].argmax()

1735

In [16]:
filtered_flights[1735].map_leaflet()

Map(center=[60.073045192193455, 9.19021778622149], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [11]:
data = filtered_flights[197].data.copy()

In [12]:
np.unwrap(np.deg2rad(data["heading"]), period=2 * np.pi, discont=np.pi).max()

7.014789267047185

In [18]:
(data["timestamp"].max() - data["timestamp"].min()).total_seconds()

2830.0

In [21]:
data["velocity"].min()

65.93117194134197

In [22]:
data

,timestamp,icao24,latitude,longitude,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,serials,hour
13947516,2023-11-03 16:12:36+00:00,45ac35,60.287109,5.220211,85.082568,170.252910,11.70432,SAS33R,False,False,False,None,381.00,129.54,1.699028e+09,1.699028e+09,[-1408232008],2023-11-03 16:00:00+00:00
13947517,2023-11-03 16:12:37+00:00,45ac35,60.286405,5.220468,85.082568,170.252910,12.35456,SAS33R,False,False,False,None,396.24,137.16,1.699028e+09,1.699028e+09,"[-1408232008, -1408233365]",2023-11-03 16:00:00+00:00
13947518,2023-11-03 16:12:38+00:00,45ac35,60.285288,5.220860,85.082568,170.252910,13.32992,SAS33R,False,False,False,None,411.48,144.78,1.699028e+09,1.699028e+09,"[-1408232008, -1408233365]",2023-11-03 16:00:00+00:00
13947519,2023-11-03 16:12:39+00:00,45ac35,60.285288,5.220860,84.575595,170.193907,13.98016,SAS33R,False,False,False,None,411.48,152.40,1.699028e+09,1.699028e+09,"[-1408232008, -1408233365]",2023-11-03 16:00:00+00:00
13947520,2023-11-03 16:12:40+00:00,45ac35,60.283813,5.221347,84.068713,170.134193,14.63040,SAS33R,False,False,False,None,441.96,190.50,1.699028e+09,1.699028e+09,"[-1408233374, -1408232008, -1408233365]",2023-11-03 16:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13950342,2023-11-03 16:59:42+00:00,45ac35,60.167953,11.103295,66.425060,16.189206,-3.57632,SAS33R,False,False,False,None,487.68,304.80,1.699031e+09,1.699031e+09,[-1408233889],2023-11-03 16:00:00+00:00
13950343,2023-11-03 16:59:43+00:00,45ac35,60.168814,11.103843,66.425060,16.189206,-3.25120,SAS33R,False,False,False,None,480.06,304.80,1.699031e+09,1.699031e+09,[-1408233889],2023-11-03 16:00:00+00:00
13950344,2023-11-03 16:59:44+00:00,45ac35,60.169418,11.104147,66.425060,16.189206,-3.25120,SAS33R,False,False,False,None,480.06,289.56,1.699031e+09,1.699031e+09,[-1408233889],2023-11-03 16:00:00+00:00
13950345,2023-11-03 16:59:45+00:00,45ac35,60.169978,11.104431,65.931172,16.313852,-3.25120,SAS33R,False,False,False,None,472.44,281.94,1.699031e+09,1.699031e+09,[-1408233889],2023-11-03 16:00:00+00:00


In [19]:
min_velocity = data["velocity"].min()

In [37]:
import pandas as pd
from typing import Tuple
from functions.data_filtering import ICAO_codes
from functions.data_loading import query
from datetime import timedelta


def get_data_range(origin: str, destination: str, start: datetime, stop: datetime) -> Tuple[Traffic, str]:
    data = []
    days = (stop - start).days
    if days <= 0:
        raise ValueError("The stop date must be after the start date.")

    for date in (start + timedelta(days=n) for n in range(days)):
        result = query.history(
            start=date,
            stop=date + timedelta(days=1),
            departure_airport=ICAO_codes[origin],
            arrival_airport=ICAO_codes[destination])
        if result is not None:
            data.append(result.copy())

    # combine data
    result = pd.concat(data, axis="rows", ignore_index=True)

    # make flight object
    final = result.rename(columns={'time': 'timestamp', 'lat': 'latitude', 'lon': 'longitude'})
    flights = Traffic(final)
    flights.data = flights.data.dropna(subset=["heading"])
    flights.resample("1s")
    
    return flights

In [38]:
f = get_data_range("oslo", "bergen", datetime(2023, month=1, day=1), datetime(year=2023, month=1, day=8))

In [39]:
np.argwhere(np.isnan(f.data["heading"]))

array([], shape=(0, 1), dtype=int64)

In [36]:
f.data.iloc[223213]

timestamp        2023-01-06 10:30:31+00:00
icao24                              45ac32
latitude                         60.208466
longitude                        11.087005
velocity                              <NA>
heading                               <NA>
vertrate                              <NA>
callsign                            SAS257
onground                             False
alert                                False
spi                                  False
squawk                                <NA>
baroaltitude                        472.44
geoaltitude                           <NA>
lastposupdate               1673001029.407
lastcontact                 1673001029.407
serials                      [-1408233889]
hour             2023-01-06 10:00:00+00:00
Name: 223213, dtype: object

In [31]:
f.data

,timestamp,icao24,latitude,longitude,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,serials,hour
0,2023-01-01 16:00:02+00:00,47878d,60.258133,8.264655,215.554492,269.726514,-0.32512,NOZ620,False,False,False,2547,9753.6,9334.5,1672588801.485,1672588801.697,[-1408232008],2023-01-01 16:00:00+00:00
1,2023-01-01 16:00:03+00:00,47878d,60.258133,8.260772,215.040053,269.72586,-0.32512,NOZ620,False,False,False,2547,9745.98,9326.88,1672588802.467,1672588802.893,[-1408232008],2023-01-01 16:00:00+00:00
2,2023-01-01 16:00:04+00:00,47878d,60.258133,8.256889,215.040053,269.72586,-0.32512,NOZ620,False,False,False,2547,9745.98,9326.88,1672588803.573,1672588803.573,[-1408232008],2023-01-01 16:00:00+00:00
3,2023-01-01 16:00:05+00:00,47878d,60.258133,8.252627,215.040053,269.72586,0.0,NOZ620,False,False,False,2547,9745.98,9326.88,1672588804.593,1672588804.926,[-1408232008],2023-01-01 16:00:00+00:00
4,2023-01-01 16:00:06+00:00,47878d,60.258101,8.249163,214.528699,269.587807,0.0,NOZ620,False,False,False,2547,9745.98,9326.88,1672588805.536,1672588805.536,[-1408232008],2023-01-01 16:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272533,2023-01-07 15:09:15+00:00,4ac9ec,60.318479,5.209678,60.139362,169.651316,-3.57632,SAS269,False,False,False,0743,320.04,220.98,1673104154.946,1673104154.946,"[-1408232008, -1408233365]",2023-01-07 15:00:00+00:00
272534,2023-01-07 15:09:16+00:00,4ac9ec,60.318479,5.209678,60.555981,170.217593,-3.90144,SAS269,False,False,False,0743,320.04,190.5,1673104154.946,1673104155.819,"[-1408232008, -1408233365]",2023-01-07 15:00:00+00:00
272535,2023-01-07 15:09:17+00:00,4ac9ec,60.317641,5.209972,60.555981,170.217593,-3.90144,SAS269,False,False,False,0743,312.42,190.5,1673104156.458,1673104156.458,"[-1408232008, -1408233365]",2023-01-07 15:00:00+00:00
272536,2023-01-07 15:09:18+00:00,4ac9ec,60.317129,5.21007,60.555981,170.217593,-2.92608,SAS269,False,False,False,0743,312.42,182.88,1673104157.442,1673104157.653,"[-1408232008, -1408233365]",2023-01-07 15:00:00+00:00
